<a href="https://colab.research.google.com/github/jalbury/machine-learning/blob/master/HW_4/HW4_Problem1_John_Albury.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [0]:
import numpy as np

# Problem 1 Description

In this problem, we are asked to implement a function `conv2d` that takes as input an input matrix (`input_mat`) and a kernel matrix (`kernel_mat`) and outputs the result of the convolution (`output_mat`). Valid padding and a stride of 1 are used.

All given matrices must be square (if either given matrix is not, an exception is thrown). If the given input matrix is smaller than the given kernel matrix, an exception is thrown.

To perform the dot product of two matrices when performing convolution, I used the `np.tensordot()` function.

# Solution

## Implementation

In [0]:
# Custom exception to handle a non-square matrix being given
class NonSquareMatrixError(Exception):
  def __init__(self, name, shape):
    self.name = name
    self.shape = shape
  def __str__(self):
    return "{} is not a square matrix (has shape {})".format(self.name,
                                                             self.shape)

# Custom exception to handle when the given kernel matrix is larger than the
# given input matrix
class InvalidConvolutionError(Exception):
  def __init__(self, input_shape, kernel_shape):
    self.input_shape = input_shape
    self.kernel_shape = kernel_shape
  def __str__(self):
    return "Kernel matrix (shape {}) is larger than input matrix (shape {})".\
            format(self.kernel_shape, self.input_shape)

# Checks that the given array is a square matrix. If it is not, a custom
# exception is thrown.
def is_square_matrix(mat, mat_name):
  if len(mat.shape) != 2 or mat.shape[0] != mat.shape[1]:
    raise NonSquareMatrixError(mat_name, mat.shape)

# Computes the convolution of a given input matrix using a given kernel matrix
def conv2d(input_mat, kernel_mat):
  # Make sure that both given arrays are square matrices
  is_square_matrix(input_mat, 'Input matrix')
  is_square_matrix(kernel_mat, 'Kernel matrix')

  # Make sure that input matrix is at least as large as the kernel matrix
  if kernel_mat.shape[0] > input_mat.shape[0]:
    raise InvalidConvolutionError(input_mat.shape, kernel_mat.shape)

  # Get number of rows/columns in input matrix
  input_size = input_mat.shape[0]

  # Get offsets from center cell that will be used when creating each patch
  if kernel_mat.shape[0] % 2 == 0:
    conv_top_left_offset = 0
    conv_bottom_right_offset = kernel_mat.shape[0] - 1
  else:
    conv_top_left_offset = int(kernel_mat.shape[0] / 2)
    conv_bottom_right_offset = conv_top_left_offset

  # Create output matrix for storing convolution result
  output_size = input_size - (conv_top_left_offset + conv_bottom_right_offset)
  output_mat = np.empty((output_size, output_size))

  # Use sliding window to get every possible patch in input matrix and fill up
  # cells in output matrix with result of convolution
  for i in range(input_size):
    for j in range(input_size):
      # Get bounds (in terms of input matrix rows and columns) for this patch
      min_patch_row = i - conv_top_left_offset
      max_patch_row = i + conv_bottom_right_offset
      min_patch_col = j - conv_top_left_offset
      max_patch_col = j + conv_bottom_right_offset

      # If patch of input matrix would include indices that are out of bounds,
      # skip this patch
      if (min_patch_row < 0 or min_patch_col < 0 or
          max_patch_row >= input_size or max_patch_col >= input_size):
        continue
      
      # Get patch from input matrix
      patch = input_mat[min_patch_row : max_patch_row + 1, 
                        min_patch_col : max_patch_col + 1]

      # Compute value in output matrix as patch "dot" kernel matrix
      output_mat[i-conv_top_left_offset, j-conv_top_left_offset] = \
                np.tensordot(patch, kernel_mat)
  
  return output_mat

## Valid matrices given

In [57]:
input_mat = np.array([[1, 2, 3, 4],
                      [5, 6, 7, 8],
                      [9, 10, 11, 12],
                      [13, 14, 15, 16]])
kernel_mat = np.array([[1, 0, 1],
                       [0, -1, 0],
                       [0, 1, 0]])

output_mat = conv2d(input_mat, kernel_mat)

print(output_mat)

[[ 8. 10.]
 [16. 18.]]


## Non-square matrix given



In [58]:
input_mat = np.array([[1, 2, 3, 4],
                      [5, 6, 7, 8],
                      [9, 10, 11, 12]])
kernel_mat = np.array([[1, 0, 1],
                       [0, -1, 0],
                       [0, 1, 0]])

output_mat = conv2d(input_mat, kernel_mat)

print(output_mat)

NonSquareMatrixError: ignored

In [59]:
input_mat = np.array([[1, 2, 3, 4],
                      [5, 6, 7, 8],
                      [9, 10, 11, 12],
                      [13, 14, 15, 16]])
kernel_mat = np.array([[1, 0],
                       [0, -1],
                       [0, 1]])

output_mat = conv2d(input_mat, kernel_mat)

print(output_mat)

NonSquareMatrixError: ignored

## Invalid matrix pair given (kernel matrix larger than input matrix)

In [60]:
input_mat = np.array([[1, 0, 1],
                       [0, -1, 0],
                       [0, 1, 0]])
kernel_mat = np.array([[1, 2, 3, 4],
                      [5, 6, 7, 8],
                      [9, 10, 11, 12],
                      [13, 14, 15, 16]])

output_mat = conv2d(input_mat, kernel_mat)

print(output_mat)

InvalidConvolutionError: ignored

## TA Test Cases

Below are the results of running my code on the test cases provided for Problem 1.

In [61]:
from scipy import signal
import numpy as np

input_mat = []
kernel_mat = []
expected_mat = []

# test case 1
input_mat.append(np.array([[1, 2, 1, 2],
                      [2, 1, 2, 1],
                      [1, 2, 1, 2],
                      [2, 1, 2, 1]]))

kernel_mat.append(np.array([[1, 0],
                       [0, 1]]))

expected_mat.append(np.array([[2, 4, 2],
                                [4, 2, 4],
                                [2, 4, 2]]))

# test case 2
input_mat.append(np.array([[1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]]))
kernel_mat.append(np.array([[1, 0], [0, 1]]))
expected_mat.append(np.array([[2, 0, 0], [0, 2, 0], [0, 0, 2]]))


# test case 3
input_mat.append(np.array([[1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]]))
kernel_mat.append(np.array([[1, -1],
                       [-1, 0]]))

expected_mat.append(np.array([[ 1, -1,  0], [-1,  1, -1],[ 0, -1,  1]]))


# test case 4
input_mat.append(np.array([[1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]]))
kernel_mat.append(np.array([[1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]]))

expected_mat.append(np.array([[4]]))


# test case 5 - should either through an error, or return empty matrix
input_mat.append(np.array([[1, -1],
                       [-1, 0]]))
kernel_mat.append(np.array([[1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]]))

expected_mat.append([])



for i in range(len(input_mat)):
  # uncomment line for student code testing
  output_mat = conv2d(input_mat[i], kernel_mat[i])

  # uncomment lines below (and comment line above) for generating test cases.
  # if input_mat[i].shape[0] < kernel_mat[i].shape[0]:
  #   output_mat = []
  # else:
  #   output_mat = signal.convolve2d(input_mat[i], kernel_mat[i], mode='valid')

  print(output_mat)
  if np.array_equal(output_mat, expected_mat[i]):
    print("Correct output!\n")
  else:
    print("Incorrect output!\n")

[[2. 4. 2.]
 [4. 2. 4.]
 [2. 4. 2.]]
Correct output!

[[2. 0. 0.]
 [0. 2. 0.]
 [0. 0. 2.]]
Correct output!

[[ 1. -1.  0.]
 [-1.  1. -1.]
 [ 0. -1.  1.]]
Correct output!

[[4.]]
Correct output!



InvalidConvolutionError: ignored